In [1]:
import xarray as xr
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas
import regionmask
from eofs.xarray import Eof
import pymc as pm
import pytensor.tensor as pt
import arviz as az
import glob
import seaborn as sns
sns.set_style("white")

import cartopy.feature as cfeature
import matplotlib as mpl

import statsmodels.api as sm
np.random.seed(123)

# Training and test periods
Define the preindustrial period (for training) and recent historical period (for testing)

In [2]:
pi_period=(800,1850)
recent_period=(1850,2000)

# SST indices
SST indices from the PHYDA dataset, downloaded and converted to csv by Ben Cook

In [ ]:
sst_fit=np.zeros((4,1201,100))
indices=["nino","amo","pdo","gmt"]
i=0
for index in indices:
    fname=glob.glob("../DATA/BenPhyda/ensphyda/indices/ensphyda*"+index+"*")[0]
    csvdata=pd.read_csv(fname)
    year=csvdata.values[:,0]
    data=csvdata.values[:,1:]
    
    if index =="gmt":
        twentiethcenturymean=np.average(data[-101:],axis=0)[np.newaxis,:]
        twentiethcenturystd=np.std(data[-101:],axis=0)[np.newaxis,:]
        data=(data-twentiethcenturymean)/twentiethcenturystd
    sst_fit[i]=data
        
    i+=1
sst_fit=xr.DataArray(sst_fit,dims=("index","year","ens"),\
                     coords={"index":indices,"year":year,"ens":np.arange(100)+1})
    #sst_fit[i]=xr.DataArray(data,dims=("year","ens"),coords={"year":year,"ens":np.arange(100)+1})
    

In [4]:
fname="../DATA/BenPhyda/ensphyda/soilmoist/ensphyda_swna_pdsi_scaled_detrend.csv"
csvdata=pd.read_csv(fname)
year=csvdata.values[:,0]
data=csvdata.values[:,1:]
soilmoisture=xr.DataArray(data,dims=("year","ens"),coords={"year":year,"ens":np.arange(100)+1})

# ACCOUNTING FOR UNCERTAINTY IN THE ENSEMBLE

In [8]:
pi_period=(800,1850)
recent_period=(1850,2000)
indices=["nino","amo","pdo","gmt"]

# Specify priors
nlags=2
priors={
    "lag_coefs": {"mu":np.tile(0,nlags),"sigma":np.tile(1,nlags),"size":nlags},
    "sigma": 10,
    "init":{"mu":0,"sigma":0.1,"size":1},
    "beta":{"amo":{"mu":0,"sigma":10},\
            "nino":{"mu":0,"sigma":10},\
           "gmt":{"mu": 0 ,"sigma":10},\
            "pdo":{"mu": 0 ,"sigma":10}}
}
#initialize the model

with pm.Model() as AR_ens:
    pass

#year interval for preindustrial data
pi_year=soilmoisture.sel(year=slice(*pi_period)).year.values

#add coordinates
AR_ens.add_coord("year",pi_year,mutable=True)
AR_ens.add_coord("lags",-1*(np.arange(nlags)+1))
#AR.add_coord("index",indices)

year_recent = np.arange(recent_period[0],recent_period[1]+1)
year_recent_lags = np.arange(recent_period[0]-nlags,recent_period[1]+1)

    ## We need to have coords for the observations minus the lagged term to correctly centre the prediction step
AR_ens.add_coords({"year_recent_lags": year_recent_lags})
AR_ens.add_coords({"year_recent": year_recent})

with AR_ens:
    SST={}
    SST_recent={}
    
    beta={}
    response={}
    for index in indices:
        data=sst_fit.sel(index=index,year=slice(*pi_period))
        meandata=data.mean(dim="ens").values
        stddata=data.std(dim="ens").values
        #prior on SST
        SST[index] = pm.Normal(index, mu=0, sigma=10, shape=len(meandata),dims="year") 
        likelihood_SST = pm.Normal(index+'_ensemble', mu=SST[index], sigma=stddata, observed=meandata)
        # Also estimate recent data, but don't use it to estimate beta
        rdata=sst_fit.sel(index=index,year=slice(*recent_period))
        rmeandata=rdata.mean(dim="ens").values
        rstddata=rdata.std(dim="ens").values
        #prior on SST recent
        SST_recent[index] = pm.Normal(index+"_recent", mu=0, sigma=10, shape=len(rmeandata),dims="year_recent") 
        likelihood_SST_recent = pm.Normal(index+'_ensemble_recent', mu=SST_recent[index], sigma=rstddata, observed=rmeandata)
        #SST[index]=pm.Deterministic(index,srng.normal(loc=SSTmu[index],scale=SSTsigma[index],size=len(data.year.values)))
        
        #Old code
        # SST[index]=pm.MutableData(index,\
        #                 sst_fit.mean(dim="ens").sel(index=index,year=slice(*pi_period)).values,\
        #                  dims="year")
        beta[index]=pm.Normal("beta_"+index,priors["beta"][index]["mu"],priors["beta"][index]["sigma"])
        response[index]=pm.Deterministic("response_to_"+index,beta[index]*SST[index],dims="year")
    
    smmean = pm.MutableData("smmean",soilmoisture.mean(dim="ens").sel(year=slice(*pi_period)).values,dims="year")
    smsigma = pm.MutableData("smsigma",soilmoisture.std(dim="ens").sel(year=slice(*pi_period)).values,dims="year")
    lag_coefs = pm.Normal("lag_coefs", priors["lag_coefs"]["mu"], priors["lag_coefs"]["sigma"],dims=("lags"))
    sigma = pm.HalfNormal("sigma", priors["sigma"])
    # We need one init variable for each lag, hence size is variable too
    init = pm.Normal.dist(
        priors["init"]["mu"], 
        priors["init"]["sigma"], 
        size=priors["init"]["size"]
    )
    # Steps of the AR model minus the lags required
    ar_n = pm.AR(
        "ar",
        lag_coefs,
        sigma=sigma,
        init_dist=init,
        constant=False,
        steps=pi_year.shape[0] - (priors["lag_coefs"]["size"]),
        dims="year",
    )
    
   
    mean = pm.math.sum(pm.math.stack([response[index] for index in indices]),axis=0) + ar_n
    smtrue=pm.Normal("pi_sm_true", mu=mean,sigma=sigma,dims=("year"))
    # Likelihood of observing preindustrial soil moisture given 
    soilmoisture_obs = pm.Normal("pi_sm",mu=smtrue,sigma=smsigma,observed = smmean,dims=("year"))
    idata_ar_ens = pm.sample_prior_predictive()
    idata_ar_ens.extend(pm.sample(2000, random_seed=100, target_accept=0.95))
    idata_ar_ens.extend(pm.sample_posterior_predictive(idata_ar_ens))

Sampling: [amo, amo_ensemble, amo_ensemble_recent, amo_recent, ar, beta_amo, beta_gmt, beta_nino, beta_pdo, gmt, gmt_ensemble, gmt_ensemble_recent, gmt_recent, lag_coefs, nino, nino_ensemble, nino_ensemble_recent, nino_recent, pdo, pdo_ensemble, pdo_ensemble_recent, pdo_recent, pi_sm, pi_sm_true, sigma]
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [nino, nino_recent, beta_nino, amo, amo_recent, beta_amo, pdo, pdo_recent, beta_pdo, gmt, gmt_recent, beta_gmt, lag_coefs, sigma, ar, pi_sm_true]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 33 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
Sampling: [amo_ensemble, amo_ensemble_recent, gmt_ensemble, gmt_ensemble_recent, nino_ensemble, nino_ensemble_recent, pdo_ensemble, pdo_ensemble_recent, pi_sm]


# SST relationships
Warm NINO = wet soil
Warm PDO = wet soil
Warm GMT = dry soil
AMO = unclear

In [11]:
    
#predict recent soil moisture from n lags: get a year series
#smrecent_plus_lags=soilmoisture.sel(year=slice(recent_period[0]-nlags,recent_period[1]))
year_recent = np.arange(recent_period[0],recent_period[1]+1)
year_recent_lags = np.arange(recent_period[0]-nlags,recent_period[1]+1)
with AR_ens:
   
    response_recent={}
    for index in indices:
       
        response_recent[index]= pm.Deterministic(index+"_response_recent",beta[index]*SST_recent[index],dims="year_recent")
    
    # condition on the learned values of the AR process
    # initialize the future AR process at *exactly* the last observed value in the AR process using a Dirac delta
    
    ar_recent = pm.AR(
        "ar_recent",
        init_dist=pm.DiracDelta.dist(ar_n[..., -nlags]),
        rho=lag_coefs,
        sigma=sigma,
        constant=False,
        dims="year_recent_lags",
    )
    
    recentmean = ar_recent[nlags:]+ pm.math.sum(pm.math.stack([response_recent[index] for index in indices]),axis=0)
    recent_sm = pm.Normal("recent_sm", mu=recentmean, sigma=sigma, dims="year_recent")
    # use the updated values and predict outcomes and probabilities:
    sm_preds = pm.sample_posterior_predictive(
        idata_ar_ens, var_names=["pi_sm", "recent_sm"]+[index+"_response_recent" for index in indices], predictions=True, random_seed=100
    )

Sampling: [ar_recent, pi_sm, recent_sm]


In [27]:
sm_preds.to_netcdf("predictions.nc",engine="h5netcdf",groups=["predictions"])

'predictions.nc'

In [14]:
idata_ar_ens.to_netcdf("posteriors.nc",groups=["posterior","posterior_predictive"])

'posteriors.nc'